In [1]:
import numpy as np
import pandas as pd
import os

### 1. Load BLS raw data

In [2]:
years = ['1990', '2000', '2010', '2020']
files = []

# create folder of all raw files
for year in years:
    folder = sorted(os.listdir('../BLS_raw/' + year + '.annual.by_industry'))
    # create list of 5 digit NAICS file names
    for file in folder[1:]:
        if len(file.split(' ')[1]) == 5:
            files.append(file)
        else:
            pass

In [3]:
# load naics codes needed for manuscript
k5 = pd.read_csv('../kurt_5digit_codes.csv')
k5 = k5['NAICS_code'].astype(str)

In [4]:
# retain only industries of interest
codes = k5
files2 = []

for file in files:
    for code in codes:
        if code in file[12:17]:
            files2.append(file)
        else:
            pass

In [5]:
df_list = []
 
# loop through year and file to append dataframes into a list
for year in years:
    for file in files2:
        if year in file:
            temp_df = pd.read_csv('../BLS_raw/' + year + '.annual.by_industry/' + file)
            mask_non50 = ((temp_df['area_fips'].str[-3:] == '000') | 
                          (temp_df['area_fips'].str[:1] == 'C') | 
                          (temp_df['area_fips'].str[:2] == 'US') | 
                          (temp_df['area_fips'].str[:1] == '7'))
            temp_df = temp_df[~mask_non50] # apply Kurt's filter within loop
            
            # add industry column name to df
            if file[:4] == '2020':
                temp_df['name'] = file[30:-4]
            else:
                temp_df['name'] = file[18:-4]
            
            df_list.append(temp_df)
        else:
            pass

# store dataframes in dictionary
names = []
for x in range(0, len(files2)):
    names.append('df' + files2[x][12:17]+ '_' + files2[x][:4])

d = dict(zip(names, df_list))

### 2. Process raw data

In [6]:
# drop undefined counties
for key in d.keys():
    d[key] = d[key][d[key]['area_fips'].str[-3:] != '999']
    #print(d[key].shape)

In [7]:
# combine public and private sector employment within-county
for key in d.keys():
    d[key] = (d[key].groupby(by=['name', 'area_fips', 'area_title'], as_index=False)['annual_avg_emplvl'].sum())

In [8]:
# load typology (US Census geographical classifications) and regional data
reg = pd.read_csv('typology/regions.csv')
typ = pd.read_csv('typology/typology.csv')

# change county codes to proper fips
typ['fips'] = typ['fips'].astype(str).str.zfill(5)

# merge BLS data with '03, '13, & '20' geographical classifications, retaining all counties from typ file
for key in d.keys():
    d[key] = d[key].merge(typ, how='left', # no undefined counties in typ, so can do outer merge to preserve
                          left_on='area_fips', right_on='fips')

# assign regions to BLS data
for key in d.keys():
    for k, v in dict(zip(reg.STATE, reg.REGION)).items():
        d[key].loc[d[key]['State'] == k, ['region']] = v
    d[key].loc[d[key]['area_title'] == 'District of Columbia', 'region'] = 'Mid-Atlantic' # give DC a region
    
# export selected columns
#final_cols = ['name', 'fips', 'area_fips', 'County Title', 'State', 'region',
#              'type_census03', 'type_bls13', 'type_census20', 'type_kurt20',
#              'annual_avg_emplvl']

final_cols = ['name', 'fips', 'County Title', 'State', 'region',
              'type_kurt20', 'annual_avg_emplvl']

for key in d.keys():
    d[key][final_cols].to_csv('my_naics5/naics_' + key[2:] + '.csv', index_label=False)

### 3. Calculate employment change columns

In [9]:
# load new NAICS CSVs
folder = sorted(os.listdir('my_naics5'))
uniquecodes = pd.Series(folder[1:]).str[6:11].unique()
path = 'my_naics5/naics_'

# use big loop to update all previously processed NAICS files

# capture the incomplete sets
missing_years = []
weird_years = []

for code in uniquecodes:
    try:
        df90 = pd.read_csv(path + code + '_1990.csv')
    except:
        missing_years.append([code, '1990'])
    try:
        df00 = pd.read_csv(path + code + '_2000.csv')
    except:
        missing_years.append([code, '2000'])
    try:    
        df10 = pd.read_csv(path + code + '_2010.csv')
    except:
        missing_years.append([code, '2010'])
    try:    
        df20 = pd.read_csv(path + code + '_2020.csv')
    except:
        missing_years.append([code, '2020'])

    # merge years under NAICS code
    temp1 = df90.merge(df00, how='outer', left_on='fips', right_on='fips', suffixes=['_90', '_00'])
    temp2 = df10.merge(df20, how='outer', left_on='fips', right_on='fips', suffixes=['_10', '_20'])
    df = temp1.merge(temp2, how='outer', left_on='fips', right_on='fips')

In [10]:
# load new NAICS CSVs
folder = sorted(os.listdir('my_naics5'))
uniquecodes = pd.Series(folder[1:]).str[6:11].unique()
path = 'my_naics5/naics_'

# use big loop to update all previously processed NAICS files

# capture the incomplete sets
missing_years = []

for code in uniquecodes:
    try:
        df90 = pd.read_csv(path + code + '_1990.csv')
    except:
        missing_years.append([code, '1990'])
    try:
        df00 = pd.read_csv(path + code + '_2000.csv')
    except:
        missing_years.append([code, '2000'])
    try:    
        df10 = pd.read_csv(path + code + '_2010.csv')
    except:
        missing_years.append([code, '2010'])
    try:    
        df20 = pd.read_csv(path + code + '_2020.csv')
    except:
        missing_years.append([code, '2020'])
        
    #df10.loc[df10['fips'].isna() & df10['area_fips'].notna(), 'fips'] = df10['area_fips']
    #df20.loc[df20['fips'].isna() & df20['area_fips'].notna(), 'fips'] = df20['area_fips']
    
    # merge years under NAICS code
    temp1 = df90.merge(df00, how='outer', left_on='fips', right_on='fips', suffixes=['_90', '_00'])
    temp2 = df10.merge(df20, how='outer', left_on='fips', right_on='fips', suffixes=['_10', '_20'])
    df = temp1.merge(temp2, how='outer', left_on='fips', right_on='fips')
    
    # fill in missing variables
    cols = ['name_90', 'County Title_90', 'State_90', 'region_90', 'type_kurt20_90']
    for col in cols:
        df[col] = df[col].fillna(df[col[:-3] + '_20'])
        df[col] = df[col].fillna(df[col[:-3] + '_10'])
        df[col] = df[col].fillna(df[col[:-3] + '_00'])
    
    # clean column names
    cols = ['name_90', 'County Title_90', 'State_90',
            'region_90', 'type_kurt20_90'] + df.filter(regex='annual').columns.tolist()
    df = df[cols]
    df.columns = df.columns.str.replace('_90', '')
    df = df.rename(columns={'annual_avg_emplvl': 'annual_avg_emplvl_90'})
    
    # replace nulls with zeroes 
    empl_cols = df.columns[-4:]
    df[empl_cols] = df[empl_cols].fillna(0)

    # rate of change function
    def rate_chg(df, year1, year2, chg):
        df[chg] = np.where((df[year1]== 0),
                          ((df[year2] - df[year1]) / 1).round(4),
                          ((df[year2] - df[year1]) / df[year1]).round(4))
    
    # define new column namer
    namer = empl_cols.str.split('_')

    # calculate rate of change
    for x in range(0,3):
        rate_chg(df, empl_cols[x], empl_cols[x+1], 'chg_' + namer[x][2] + '_' + namer[x][3] + '_' + namer[x+1][3])
    
    # calculate total rate of change column (1990-2020)
    df['chg_emplvl_90_20'] = np.where((df['annual_avg_emplvl_90']==0),
                                      ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / 1).round(4),
                                      ((df['annual_avg_emplvl_20'] - df['annual_avg_emplvl_90']) / df['annual_avg_emplvl_90']).round(4))
    
    # store industry name
    name_probs = []
    try:
        industry_name = df['name'][df['name'].notna()][0]
    except:
        name_probs.append(code)
    
    # export file
    # print(code, df[df['County Title'].notna()].shape)
    df.to_csv('my_naics5_chg/naics_' + code + '_' + industry_name + '.csv',
               index_label=False)